In [ ]:
movies_with_revenue = movie_df[movie_df.revenue != 0]
movies_with_revenue = pd.merge(movies_with_revenue, similarity_df, on='name', how='inner')
movies_with_revenue = movies_with_revenue.drop(['release_date_y', 'genres_y', 'plot_y', 'freebase_id', 'runtime', 'genre_divided'], axis=1)
movies_with_revenue = movies_with_revenue.rename(columns={'release_date_x': 'release_date', 'genres_x': 'genres', 'plot_x': 'plot'})

In [ ]:
def load_inflation_data():
    file_path = "./inflation_data/API_FP/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_4763429.csv"
    return pd.read_csv(file_path, sep=',', header=2)

In [ ]:
def get_inflation_list():
    """
    First year in inflation_list is 1960, last year is 2021
    """
    inflation_df = load_inflation_data()
    inflation_df = inflation_df.drop(columns='Unnamed: 66')
    inflation_df = inflation_df.dropna(axis=0, how='any')
    inflation_df = inflation_df[inflation_df['Country Code'] == 'USA']
    inflation_list = inflation_df.loc[inflation_df['Country Code'] == 'USA'].values.flatten().tolist()
    for i in range(4):
        inflation_list.pop(0)
    return inflation_list
    
inflation_list = get_inflation_list()

In [ ]:
def get_adjusted_revenue(x, inflation_list):
    release_date = x['release_date']
    start_idx = release_date - 1960
    if start_idx < 0:
        return None
    adj_revenue = x['revenue']
    for idx in range(start_idx, len(inflation_list)):
        adj_revenue = inflation_list[idx]/100 * adj_revenue + adj_revenue

    return adj_revenue

In [ ]:
movies_with_revenue['adj_revenue'] = movies_with_revenue.apply(lambda x : get_adjusted_revenue(x, inflation_list), axis=1)

In [ ]:
movies_with_revenue = movies_with_revenue[movies_with_revenue.adj_revenue.notna()]

In [ ]:
event_list = movies_with_revenue.event.unique()

In [ ]:
for i in range(len(event_list)):
    event_df = movies_with_revenue[movies_with_revenue.event == event_list[i]]
    event_df.plot.scatter(x='similarity', y='adj_revenue', s=0.8)
    plt.title(event_list[i])
    plt.show()

In [ ]:
adjrev_by_event = movies_with_revenue.groupby(by='event')[['adj_revenue']].mean()

In [ ]:
adjrev_by_event.plot.bar(y='adj_revenue')

In [ ]:
movies_with_revenue['is_after'] = movies_with_revenue.apply(lambda x: 1 if x['release_date'] > extract_date_interval(dates_df.loc[x['event']]['date'])[0] else 0, axis=1)

In [ ]:
movies_with_revenue.sort_values(by='release_date', ascending=True)

In [ ]:
f = plt.figure()
sns.barplot(data = movies_with_revenue, x='event', y = 'revenue', hue='is_after', palette='turbo')
plt.xticks(rotation=90)
# plt.figure(figsize=(32, 20), dpi=80)
f.set_figheight(12)
f.set_figwidth(12)